# 自动求导

### 一个简单的案例：
计算$y = x*x$的导数：

In [52]:
import torch

x = torch.arange(4,dtype=torch.float32, requires_grad=True) # 定义需要求导的变量, requires_grad=True表示需要存梯度
y = 2*torch.dot(x, x) # 定义需要求导的表达式
y.backward() # 计算梯度
print(x.grad) # 输出x的梯度

tensor([ 0.,  4.,  8., 12.])


第二个案例：计算$y = sum(x)$的导数

错误示范：不清零梯度，会导致梯度累积

In [53]:
y = x.sum()
y.backward()
print(x.grad)

tensor([ 1.,  5.,  9., 13.])


以下为正确案例：

In [54]:
x.grad.zero_()
y = x.sum()
y.backward()
print(x.grad)

tensor([1., 1., 1., 1.])


### 把一些计算移动到记录的计算图之外

In [55]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x
z.sum().backward() # 这里u视作常数，而非x的函数，故求导后x的梯度为u
print(x.grad == u)

tensor([True, True, True, True])


### 复杂的计算图，如循环、条件语句，仍可以自动求导

In [56]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()
print(a.grad==d/a)

tensor(True)


由于d和a的关系永远是线性正比例的，所以导数就是d/a（虽然函数很复杂）